In [4]:
import numpy as np
import pandas as pd
import scipy.optimize
import bokeh.plotting
import bokeh.io
bokeh.io.output_notebook()

Loading BokehJS ...

In [7]:
def agonist_only_p_active_theory(c_agonist, log_Kd_active, log_Kd_inactive, beta_deltaE):
    "theoretical curve for mglur5 active probability"
    #log_Kd_inactive = -7.7

    Kd_active = 10**log_Kd_active
    Kd_inactive = 10**log_Kd_inactive
    a = (1 + c_agonist/Kd_active)**2

    b = (1 + c_agonist/Kd_inactive)**2

    return a/(a+b*np.exp(-beta_deltaE))

def resid(param, c, p_active):
    return p_active - agonist_only_p_active_theory(c, *param)


In [8]:
def PAM_only_p_active_theory(c_pam, log_Kd_active, log_Kd_inactive, beta_deltaE):
    "theoretical curve for mglur5 active probability"
    beta_deltaE=-8
    Kd_active = 10**log_Kd_active
    Kd_inactive = 10**log_Kd_inactive
    a = (1 + c_pam/Kd_active)

    b = (1 + c_pam/Kd_inactive)

    return a/(a+b*np.exp(-beta_deltaE))

def resid_PAM(param, c, p_active):
    return p_active - PAM_only_p_active_theory(c, *param)


In [9]:
#set up plot
from bokeh.plotting import curdoc 
curdoc().clear()
p = bokeh.plotting.figure(plot_height=400,
                         plot_width=600,
                         x_axis_label='[L-Quis]M',
                          y_axis_label='active_probability',
                          x_axis_type='log')

colors=bokeh.palettes.d3['Category20'][20]

ligand_smooth=np.logspace(-11,-3,200)


p.line(ligand_smooth, agonist_only_p_active_theory(ligand_smooth, -10, -8, -8), color=colors[1],legend_label='-10,-8,-8')
p.line(ligand_smooth, agonist_only_p_active_theory(ligand_smooth, -15, -8, -31), color=colors[2],legend_label='-15,-8,-34')
        


bokeh.io.show(p);

In [11]:
#set up plot
from bokeh.plotting import curdoc 
curdoc().clear()
p = bokeh.plotting.figure(plot_height=400,
                         plot_width=600,
                         x_axis_label='[L-Quis]M',
                          y_axis_label='active_probability',
                          x_axis_type='log')

colors=bokeh.palettes.d3['Category20'][20]

ligand_smooth=np.logspace(-12,-4,200)

for i,deltaE in enumerate(range(-1,-12,-1)):    
    p.line(ligand_smooth, agonist_only_p_active_theory(ligand_smooth, -9.8, -7.7, deltaE), color=colors[i],legend_label='delta E='+str(deltaE))


bokeh.io.show(p);

In [148]:
#set up plot
from bokeh.plotting import curdoc 
curdoc().clear()
p = bokeh.plotting.figure(plot_height=600,
                         plot_width=600,
                         x_axis_label='[L-Quis]M',
                          y_axis_label='active_probability',
                          x_axis_type='log')

colors=bokeh.palettes.d3['Category20'][20]

ligand_smooth=np.logspace(-11,-3,200)

for i,log_Kd_inactive in enumerate(range(-3,-10,-1)):    
    p.line(ligand_smooth, agonist_only_p_active_theory(ligand_smooth, -10, log_Kd_inactive, -8), color=colors[i],legend_label='log_Kd_inactive='+str(log_Kd_inactive))



bokeh.io.show(p);

In [55]:
#set up plot
from bokeh.plotting import curdoc 
curdoc().clear()
p = bokeh.plotting.figure(plot_height=600,
                         plot_width=600,
                         x_axis_label='[L-Quis]M',
                          y_axis_label='active_probability',
                          x_axis_type='log')

colors=bokeh.palettes.d3['Category20'][20]

ligand_smooth=np.logspace(-11,-3,200)

for i,log_Kd_active in enumerate(range(-8,-16,-1)):    
    p.line(ligand_smooth, agonist_only_p_active_theory(ligand_smooth, log_Kd_active, -7.7, -6), color=colors[i],legend_label='log_Kd_active='+str(log_Kd_active))



bokeh.io.show(p);

In [120]:
df_glu = pd.read_excel('./L-glu_nomedia_curves.xlsx')



#initial parameter for optimization
p0 = np.array([-6,-5,-8])
c, p_active = df_glu['conc'].values, df_glu['p_open'].values/100

#optimize kd and energy difference to fit the data
res_glu = scipy.optimize.least_squares(resid, p0, args = (c,p_active))

curdoc().clear()
p = bokeh.plotting.figure(plot_height=300,
                         plot_width=600,
                         x_axis_label='[L-glu] M',
                          y_axis_label='active_probability',
                          x_axis_type='log')
ligand_smooth=np.logspace(-8,-4,200)

p.line(ligand_smooth, agonist_only_p_active_theory(ligand_smooth, res_glu.x[0],res_glu.x[1], res_glu.x[2]))
p.circle(c, p_active, size=7)

bokeh.io.show(p);
print(res_glu.x)

[-7.6753466 -5.6741383 -8.       ]


In [146]:
df = pd.read_excel('./L-quis_nomedia_curves.xlsx')



#initial parameter for optimization
p0 = np.array([-9,-7,-4.676])
c, p_active = df['conc'].values, df['p_open'].values/100

#optimize kd and energy difference to fit the data
res = scipy.optimize.least_squares(resid, p0, args = (c,p_active))

curdoc().clear()
p = bokeh.plotting.figure(plot_height=400,
                         plot_width=600,
                         x_axis_label='[L-Quis] M',
                          y_axis_label='active_probability',
                          x_axis_type='log')
ligand_smooth=np.logspace(-11,-4,200)

p.line(ligand_smooth, agonist_only_p_active_theory(ligand_smooth, res.x[0],res.x[1], res.x[2]), color = 'black')
p.circle(c, p_active, size=7, color = 'black')

bokeh.io.show(p);
print(res.x)

[-9.78143649 -7.72068611 -4.676     ]


In [139]:
df_pam = pd.read_excel('./CDPPB_nomedia_curves.xlsx')

#initial parameter for optimization
p0 = np.array([-4,-3,-8])
c_pam, p_active_pam = df_pam['conc'].values, df_pam['p_open'].values/100

#optimize kd and energy difference to fit the data
res_pam = scipy.optimize.least_squares(resid_PAM, p0, args = (c_pam,p_active_pam))

curdoc().clear()
p = bokeh.plotting.figure(plot_height=400,
                         plot_width=600,
                         x_axis_label='[CDPPB] M',
                          y_axis_label='active_probability',
                          x_axis_type='log')


ligand_smooth=np.logspace(-8,-2,200)
p.line(ligand_smooth, PAM_only_p_active_theory(ligand_smooth,res_pam.x[0],res_pam.x[1],res_pam.x[2]), color='red')
p.circle(c_pam, p_active_pam, size=7, color='red')

bokeh.io.show(p);
print(res_pam.x)

[-8.03944723 -4.86304141 -8.        ]


In [107]:
df_pam = pd.read_excel('./CDPPB_curves.xlsx')


#initial parameter for optimization
p0 = np.array([-4,-3,-8])
c_pam, p_active_pam = df_pam['conc'].values, df_pam['p_open'].values

#optimize kd and energy difference to fit the data
res_pam = scipy.optimize.least_squares(resid_PAM, p0, args = (c_pam,p_active_pam))

curdoc().clear()
p = bokeh.plotting.figure(plot_height=300,
                         plot_width=600,
                         x_axis_label='[CDPPB] M',
                          y_axis_label='active_probability',
                          x_axis_type='log')


ligand_smooth=np.logspace(-8,-2,200)
p.line(ligand_smooth, PAM_only_p_active_theory(ligand_smooth, res_pam.x[0],res_pam.x[1],res_pam.x[2]))
p.circle(c_pam, p_active_pam, size=7)

bokeh.io.show(p);
print(res_pam.x)

[-16.14000697  -5.46756344 -23.5366297 ]


In [143]:
def both_p_active_theory(c_agonist, c_pam, log_Ka, log_Ki, log_Kpa, log_Kpi):
    "theoretical curve for mglur5 active probability"
    beta_deltaE = -8
    Ka = 10**log_Ka
    Ki = 10**log_Ki
    Kpa = 10**log_Kpa
    Kpi = 10**log_Kpi
    
    a = (1 + c_agonist/Ka)**2

    b = (1 + c_agonist/Ki)**2
    
    c = (1 + c_pam/Kpa)

    d = (1 + c_pam/Kpi)

    return (a*c)/((a*c)+(b*d*np.exp(-beta_deltaE)))


In [123]:
res.x

array([-9.7814442 , -7.72069745, -8.        ])

In [8]:
df_both = pd.read_excel('./L-quis+CDPPB.xlsx')


#initial parameter for optimization
#p0 = np.array([-8,-4])
c_agonist, p_active_both = df_both['conc'].values, df_both[0].values/100

#optimize kd and energy difference to fit the data
#res_pam = scipy.optimize.least_squares(both_resid, p0, args = (res.x[0],res.x[1],c_agonist,4e-7, p_active_pam))

curdoc().clear()
p = bokeh.plotting.figure(plot_height=600,
                         plot_width=600,
                         x_axis_label='[L-QUIS] M',
                          y_axis_label='active_probability',
                          x_axis_type='log')


ligand_smooth=np.logspace(-11,-4,200)

p.line(ligand_smooth, both_p_active_theory(ligand_smooth, 0,res.x[0], res.x[1], res_pam.x[0], res_pam.x[1]))
p.circle(c_agonist, p_active_both, size=7)

bokeh.io.show(p);


In [151]:
df_both = pd.read_excel('./L-quis+CDPPB.xlsx')


colors=bokeh.palettes.d3['Category10'][10]


curdoc().clear()
p = bokeh.plotting.figure(plot_height=400,
                         plot_width=600,
                         x_axis_label='[L-Quis] M',
                          y_axis_label='active_probability',
                          x_axis_type='log')


ligand_smooth=np.logspace(-12,-5,200)

for i,CDPPBconc in enumerate(df_both.columns[1:]):
    c, p_active = df_both['conc'].values, df_both[CDPPBconc].values/100
    p.circle(c, p_active, size=7, color=colors[i],legend_label='CDDPB conc: '+str(CDPPBconc)+'nM')
    p.line(ligand_smooth, both_p_active_theory(ligand_smooth, CDPPBconc*1e-09, res.x[0],res.x[1], res_pam.x[0], res_pam.x[1]), color=colors[i])

p.legend.location = "top_left"
bokeh.io.show(p);

In [18]:
for i,CDPPBconc in enumerate(df_both.columns[1:]):
    print(CDPPBconc)
    print(type(CDPPBconc))

0
<class 'int'>
400
<class 'int'>
2000
<class 'int'>
10000
<class 'int'>


In [133]:
10**res_pam.x

array([2.82821387e-07, 1.58074859e-05, 2.10768706e-05])

In [138]:
10000*10**(-9)

1e-05

In [139]:
1e-05

1e-05